In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

from sklearn.preprocessing import MultiLabelBinarizer

# Загрузка Данных

In [2]:
df = pd.read_csv('main_task_new.csv')

In [3]:
df.sample(5)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
4766,id_3680,Barcelona,NaN,3681.0,4.0,NaN,17.0,"[['Friendly, great value for money.', 'Fresh a...",/Restaurant_Review-g187497-d7189327-Reviews-El...,d7189327
39453,id_4475,Paris,['Italian'],4476.0,4.0,$$ - $$$,58.0,"[['Great restaurant & excellent service', 'An ...",/Restaurant_Review-g187147-d1327148-Reviews-De...,d1327148
24405,id_5245,Rome,"['Italian', 'Pizza', 'Mediterranean']",5246.0,3.5,$$ - $$$,221.0,"[['Nice but small portions', 'Salty Pasta!'], ...",/Restaurant_Review-g187791-d2174384-Reviews-An...,d2174384
28641,id_1418,Madrid,['American'],1420.0,4.0,$$ - $$$,189.0,"[['Incredible', 'A tiny slip that could have c...",/Restaurant_Review-g187514-d9708353-Reviews-Go...,d9708353
4683,id_16174,London,['American'],16186.0,2.0,NaN,8.0,"[['Terrible staff', 'Very rude staff'], ['05/0...",/Restaurant_Review-g186338-d5122124-Reviews-Kf...,d5122124


#### основные поля

- Restaurant_id — идентификационный номер ресторана / сети ресторанов;
- City — город, в котором находится ресторан;
- Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
- Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
- Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
- Price Range — диапазон цен в ресторане;
- Number of Reviews — количество отзывов о ресторане;
- Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
- URL_TA — URL страницы ресторана на TripAdvisor;
- ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [5]:
def missing_values_table(df):
    
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Отсутствующих данных', 1 : '% от всех данных'})
        
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% от всех данных', ascending=False).round(1)
        
        print ("Было выбрано " + str(df.shape[1]) + " колонок.\n"      
            "Среди их " + str(mis_val_table_ren_columns.shape[0]) +
              " с отсутствующими данными.")
        
        return mis_val_table_ren_columns
    
missing_values_table(df)

Было выбрано 10 колонок.
Среди их 3 с отсутствующими данными.


,Отсутствующих данных,% от всех данных
Price Range,13886,34.7
Cuisine Style,9283,23.2
Number of Reviews,2543,6.4


# Очистка данных и генерация новых признаков

In [6]:
# код по очистке данных и генерации новых признаков

In [7]:
# кол-во видов кухни
def cuisine_count(value):
    if not pd.isna(value):
        return len(value.split(','))
    else:
        return 1

df['cuisine_count'] = df['Cuisine Style'].apply(cuisine_count)


# текст отзывов
def reviews_text(value):
    review = value.replace("'",'').strip('[]').split('], [')
    if len(review) > 1:
        text = review[0]
        return text
    else:
        return np.nan
    
df['reviews_text'] = df['Reviews'].apply(reviews_text)


# даты отзывов
def reviews_date(value):
    review = value.replace("'",'').strip('[]').split('], [')
    if len(review) > 1:
        dates = review[1].replace(' ', '')
        return dates
    else:
        return np.nan
    
df['reviews_dates'] = df['Reviews'].apply(reviews_date)


# кол-во дней между отзывами
def days_between(dates):
    if pd.isna(dates):
        return 0
    else:
        date_list = dates.split(',')
        if len(date_list) > 1:  
            d1 = datetime.strptime(date_list[0], '%m/%d/%Y')
            d2 = datetime.strptime(date_list[1], '%m/%d/%Y')
            return abs((d2 - d1).days)
        else:
            return 0
    
df['days_between'] = df['reviews_dates'].apply(days_between)


# наличие отзывов
def reviews_pres(review):
    if pd.isna(review):
        return 0
    else:
        return 1
    
df['reviews_pres'] = df['reviews_text'].apply(reviews_pres)


# уровень цен
def price_level(value):
    if pd.isna(value):
        return 2
    else:
        if value == '$':
            return 1
        elif value == '$$ - $$$':
            return 2
        elif value == '$$$$':
            return 3
    
df['price_level'] = df['Price Range'].apply(price_level)


# редактирование поля 'Number of Reviews'
def number_rev_ed(value):
    if pd.isna(value[0]):
        if pd.isna(value[1]):
            return 0
        else:
            return len(value[1].split(','))
    else:
        return value[0]
        
df['Number of Reviews'] = df[['Number of Reviews', 'reviews_dates']].apply(number_rev_ed, axis=1)

In [8]:
# площадь городов
city_area_dict = {
    'London': 1572,
    'Paris': 105.4,
    'Madrid': 604.3,
    'Barcelona': 101.9,
    'Berlin': 891.8,
    'Milan': 181.8,
    'Rome': 1285,
    'Prague': 496,
    'Lisbon': 100,
    'Vienna': 414.6,
    'Amsterdam': 219.3,
    'Brussels': 32.61,
    'Hamburg': 755.2,
    'Munich': 310.7,
    'Lyon': 47.87,
    'Stockholm': 188,
    'Budapest': 525.2,
    'Warsaw': 517.2,
    'Dublin': 117.8,
    'Copenhagen': 88.25,
    'Athens': 38.96,
    'Edinburgh': 264,
    'Zurich': 87.88,
    'Oporto': 41.42,
    'Geneva': 15.93,
    'Krakow': 327,
    'Oslo': 454,
    'Helsinki': 213.8,
    'Bratislava': 367.6,
    'Luxembourg': 51.46,
    'Ljubljana': 163.8,
}

def city_area(value):
    return city_area_dict.get(value)

df['city_area'] = df['City'].apply(city_area)

In [9]:
# государства
country_dict = {
    'London': 'England',
    'Paris': 'France',
    'Madrid': 'Spain',
    'Barcelona': 'Spain',
    'Berlin': 'Germany',
    'Milan': 'Italy',
    'Rome': 'Italy',
    'Prague': 'Czechia',
    'Lisbon': 'Portugal',
    'Vienna': 'Austria',
    'Amsterdam': 'Netherlands',
    'Brussels': 'Belgium',
    'Hamburg': 'Germany',
    'Munich': 'Germany',
    'Lyon': 'France',
    'Stockholm': 'Sweden',
    'Budapest': 'Hungary',
    'Warsaw': 'Poland',
    'Dublin': 'Ireland',
    'Copenhagen': 'Denmark',
    'Athens': 'Greece',
    'Edinburgh': 'Scotland',
    'Zurich': 'Switzerland',
    'Oporto': 'Portugal',
    'Geneva': 'Switzerland',
    'Krakow': 'Poland',
    'Oslo': 'Norway',
    'Helsinki': 'Finland',
    'Bratislava': 'Slovakia',
    'Luxembourg': 'st. Luxembourg',
    'Ljubljana': 'Slovenia',
}

def country(value):
    return country_dict.get(value) 

df['country'] = df['City'].apply(country)

In [10]:
# заполнение пропусков Cuisine Style
cuisine_dict = {
    'Italy': 'Italian',
    'Sweden': 'Swedish',
    'Germany': 'German',
    'Portugal': 'Portuguese',
    'Austria': 'Austrian',
    'Spain': 'Spanish',
    'France': 'French',
    'Netherlands': 'Dutch',
    'Hungary':'Hungarian',
    'Denmark':'Danish',
    'England':'British',
    'Scotland':'Scottish',
    'Slovakia':'Slovak',
    'Belgium':'Belgian',
    'Poland':'Polish',
    'Ireland':'Irish',
    'Czechia':'Czech',
    'Norway':'Norwegian',
    'Switzerland':'Swiss',
    'Greece':'Greek',
    'Slovenia': 'Slovenian',
    'st. Luxembourg': 'cs. Luxembourg',
    'Finland': 'Finnish'
}

def cuisine_ed(value):
    if pd.isna(value[0]):
        return cuisine_dict.get(value[1])
    else:
        return value[0].replace("'",'').strip('[]')
        
df['Cuisine Style'] = df[['Cuisine Style', 'country']].apply(cuisine_ed, axis=1)
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x.split(', '))

#### dummy - переменные

In [11]:
# dummy - переменные
city_dum = pd.get_dummies(df['City'])
country_dum = pd.get_dummies(df['country'])

cuisine = df['Cuisine Style']
mlb = MultiLabelBinarizer()
cuisine_dum = pd.DataFrame(mlb.fit_transform(cuisine),columns=mlb.classes_, index=df.index)

In [12]:
df = df.join(city_dum)
df = df.join(country_dum)
df = df.join(cuisine_dum)

### Корреляция

In [14]:
corr_df = df[
    [
        'Ranking', 'Rating', 'Number of Reviews',
        'cuisine_count', 'days_between',
        'reviews_pres', 'price_level'
    ]
]

corr = corr_df.corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])]


corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

,Ranking,Rating,Number of Reviews,cuisine_count,days_between,reviews_pres,price_level
Ranking,1.00,-0.37,-0.22,-0.32,0.07,-0.11,-0.03
Rating,-0.37,1.00,0.03,0.12,-0.07,0.02,-0.03
Number of Reviews,-0.22,0.03,1.00,0.41,-0.09,0.16,0.11
cuisine_count,-0.32,0.12,0.41,1.00,-0.12,0.29,0.00
days_between,0.07,-0.07,-0.09,-0.12,1.00,0.23,-0.04
reviews_pres,-0.11,0.02,0.16,0.29,0.23,1.00,-0.08
price_level,-0.03,-0.03,0.11,0.00,-0.04,-0.08,1.00


### Удаление стобцов

In [15]:
drop_col = ['City', 'Cuisine Style',
            'Price Range', 'Reviews',
            'URL_TA', 'ID_TA',
            'reviews_text',
            'reviews_dates', 'country']
df = df.drop(columns=drop_col, axis=1)

# Разбиение датафрейма на части, необходимые для обучения и тестирования модели

In [16]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [17]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создание, обучание и тестирование модели

In [18]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [19]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.210817
